In [ ]:
import requests
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('path.csv')

In [1]:
# Define the Google Distance Matrix API key
API_KEY = "YOUR_API_KEY"


# Function to fetch distance matrix
def get_distance_matrix(locations, mode="driving", avoid=None):
    """
    Fetches a full distance matrix between all pairs of locations using Google Distance Matrix API.
    :param locations: List of locations as tuples (lat, lon).
    :param mode: Mode of transport (default: 'driving').
    :param avoid: Avoid options (e.g., 'tolls', 'ferries').
    :return: A NumPy matrix of distances (in kilometers) and durations (in minutes).
    """
    # Format locations for the API
    locations_str = "|".join([f"{lat},{lon}" for lat, lon in locations])

    # Build the request URL
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        "origins": locations_str,
        "destinations": locations_str,
        "mode": mode,
        "key": API_KEY,
    }
    if avoid:
        params["avoid"] = avoid

    # Make the API request
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        return None

    # Parse the API response
    data = response.json()

    # Initialize matrices for distances (km) and durations (minutes)
    num_locations = len(locations)
    distance_matrix = np.zeros((num_locations, num_locations))
    duration_matrix = np.zeros((num_locations, num_locations))

    # Populate the matrices
    for i, row in enumerate(data["rows"]):
        for j, element in enumerate(row["elements"]):
            if element["status"] == "OK":
                distance_matrix[i][j] = (
                    element["distance"]["value"] / 1000
                )  # Convert meters to km
                duration_matrix[i][j] = (
                    element["duration"]["value"] / 60
                )  # Convert seconds to minutes
            else:
                distance_matrix[i][j] = np.inf  # Mark unreachable routes as infinity
                duration_matrix[i][j] = np.inf

    return distance_matrix, duration_matrix


# Example usage
if __name__ == "__main__":
    # List of locations as (latitude, longitude)
    locations = [
        (7.2906, 80.6337),  # Kiribathkumbura
        (7.4863, 80.3623),  # Kurunagala
        (8.2121, 80.5061),  # Rambawa
        (8.1423, 80.9713),  # Medirigiriya
    ]

    # Call the function
    distances, durations = get_distance_matrix(
        locations, mode="driving", avoid="tolls|ferries"
    )

    # Print the results
    print("Distance Matrix (km):")
    print(distances)

    print("\nDuration Matrix (minutes):")
    print(durations)

Distance Matrix (km):
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Duration Matrix (minutes):
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [2]:
import requests
import folium

# Define the Google Directions API key
API_KEY = "YOUR_API_KEY"


# Function to get the path between two locations
def get_route(origin, destination, mode="driving", avoid=None):
    """
    Fetches the route between two locations using Google Directions API.
    :param origin: Starting location as (lat, lon).
    :param destination: Ending location as (lat, lon).
    :param mode: Mode of transport (default: 'driving').
    :param avoid: Avoid options (e.g., 'tolls', 'ferries').
    :return: List of coordinates representing the path.
    """
    # Format the origin and destination
    origin_str = f"{origin[0]},{origin[1]}"
    destination_str = f"{destination[0]},{destination[1]}"

    # Build the request URL
    url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": origin_str,
        "destination": destination_str,
        "mode": mode,
        "key": API_KEY,
    }
    if avoid:
        params["avoid"] = avoid

    # Make the API request
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        return None

    # Parse the API response
    data = response.json()
    if data["status"] == "OK":
        # Decode the polyline geometry
        steps = data["routes"][0]["legs"][0]["steps"]
        path = []
        for step in steps:
            path.append((step["start_location"]["lat"], step["start_location"]["lng"]))
        path.append(
            (steps[-1]["end_location"]["lat"], steps[-1]["end_location"]["lng"])
        )
        return path
    else:
        print(f"Error: {data['status']}, {data.get('error_message', '')}")
        return None


# Function to visualize the route on a map
def visualize_route(route, origin, destination):
    """
    Visualizes the route on a map.
    :param route: List of (lat, lon) coordinates for the route.
    :param origin: Starting location as (lat, lon).
    :param destination: Ending location as (lat, lon).
    :return: Folium map.
    """
    # Create a base map centered on the origin
    m = folium.Map(location=origin, zoom_start=10)

    # Add the route as a PolyLine
    folium.PolyLine(route, color="blue", weight=5, opacity=0.7).add_to(m)

    # Add markers for origin and destination
    folium.Marker(origin, popup="Origin", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker(
        destination, popup="Destination", icon=folium.Icon(color="red")
    ).add_to(m)

    return m


# Example usage
if __name__ == "__main__":
    # Define origins and destinations
    origin = (7.2906, 80.6337)  # Kiribathkumbura
    destination = (7.4863, 80.3623)  # Kurunagala

    # Get the route
    route = get_route(origin, destination, mode="driving", avoid="tolls|ferries")

    if route:
        # Visualize the route
        map_obj = visualize_route(route, origin, destination)
        

Error: REQUEST_DENIED, The provided API key is invalid. 
